In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os
import subprocess
import time
import datetime
import re
from typing import Annotated, TypedDict, Sequence, List, Union
from pydantic import BaseModel, Field
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages

In [3]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    extracted_time: str
    script_name: str
    script_args: List[str]
    schtasks_command: str
    result: str

In [4]:
llm = ChatOpenAI(model="gpt-4o-mini")

class ScriptChoice(BaseModel):
    script_name: str = Field(description="Which script does the user want?")
    script_args: List[str] = Field(default_factory=list, description="List of arguments.")

def parse_and_format_date(iso_string: str) -> Union[str, str]:
    if iso_string.lower() == "no time found." or not iso_string.strip():
        dt = datetime.datetime.now() + datetime.timedelta(minutes=1)
    else:
        cleaned = re.sub(r"\s+[A-Z]{2,4}$", "", iso_string.strip())
        try:
            dt = datetime.datetime.strptime(cleaned, "%Y-%m-%d %H:%M")
        except ValueError:
            dt = datetime.datetime.now() + datetime.timedelta(minutes=1)
    date_str = dt.strftime("%d.%m.%Y")
    time_str = dt.strftime("%H:%M")
    return date_str, time_str

In [5]:
def time_extractor(state: AgentState) -> AgentState:
    print("[DEBUG] Entering time_extractor node...")
    start_time = time.time()
    user_message = state["messages"][-1].content
    now_berlin = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=1))).strftime("%Y-%m-%d %H:%M:%S %Z")
    system_prompt = (
        "You are a time extractor who also knows the current time in Berlin.\n"
        f"Current Berlin time is: {now_berlin}.\n\n"
        "Your job:\n"
        "1) Read the user's text for a date/time mention (like \"tomorrow in 2 minutes\").\n"
        "2) Attempt to interpret that date/time with respect to the current Berlin time you have.\n"
        "3) Return the exact final date/time in a short, standardized form (e.g. \"2025-01-18 10:32 CET\").\n"
        "If the user is ambiguous, do your best guess. If no time is mentioned, say: 'No time found.'"
    )
    prompt = ChatPromptTemplate.from_messages([("system", system_prompt), ("human", "{text}")])
    chain = prompt | llm
    result = chain.invoke({"text": user_message})
    state["extracted_time"] = result.content.strip()
    print(f"[DEBUG] time_extractor finished. LLM Output: {result.content!r}")
    print(f"[DEBUG] time_extractor duration: {time.time() - start_time:.2f} seconds\n")
    return state

In [6]:
def script_parser(state: AgentState) -> AgentState:
    print("[DEBUG] Entering script_parser node...")
    start_time = time.time()
    user_message = state["messages"][-1].content
    system_prompt = (
        "We have three scripts available:\n"
        "1) alert_script\n"
        "2) notify_script\n"
        "3) report_script\n\n"
        "You can parse any arguments the user might mention (like --verbose, or something else).\n"
        "- script_name: must be one of [alert_script, notify_script, report_script].\n"
        "- script_args: a list of strings (arguments).\n"
        "If the user doesn't mention arguments, use an empty array.\n"
        "If the user doesn't mention a specific script, default to \"alert_script\"."
    )
    structured_llm = llm.with_structured_output(ScriptChoice)
    prompt = ChatPromptTemplate.from_messages([("system", system_prompt), ("human", "{text}")])
    chain = prompt | structured_llm
    result = chain.invoke({"text": user_message})
    state["script_name"] = result.script_name
    state["script_args"] = result.script_args
    print(f"[DEBUG] script_parser finished. Structured LLM Output: {result}")
    print(f"[DEBUG] script_parser duration: {time.time() - start_time:.2f} seconds\n")
    return state

try:
    current_dir = os.path.dirname(os.path.realpath(__file__))
except NameError:
    current_dir = os.getcwd()

scripts_path = os.path.join(current_dir, "scripts")

In [7]:
def windows_crontab_translator(state: AgentState) -> AgentState:
    print("[DEBUG] Entering windows_crontab_translator node...")
    start_time = time.time()
    time_str = state.get("extracted_time", "No time found.")
    script_name = state.get("script_name", "alert_script")
    script_args = state.get("script_args", [])
    arg_str = " ".join(script_args)
    system_prompt = (
        "You are a converter for Windows Task Scheduler commands (schtasks).\n"
        "Given:\n"
        "- A final date/time expression (e.g. \"2025-01-18 10:32 CET\")\n"
        "- A script name (e.g. \"alert_script\")\n"
        "- Additional arguments to pass to that script\n\n"
        "You produce a single-line schtasks command to schedule that script at that date/time,\n"
        "including \"/f\" to force overwrite if the task already exists.\n"
        "Only return the command, nothing else.\n"
        "Example:\n"
        'schtasks /create /TN "alert_script" /SC once /ST 10:32 /SD 01/18/2025 /TR "C:\\scripts\\alert_script.bat --verbose test123" /f'
    )
    human_prompt = f"Date/Time: {time_str}\nScript: {script_name}\nArguments: {arg_str}\n"
    prompt = ChatPromptTemplate.from_messages([("system", system_prompt), ("human", "{text}")])
    chain = prompt | llm
    llm_result = chain.invoke({"text": human_prompt})
    print(f"[DEBUG] windows_crontab_translator finished. LLM Output: {llm_result.content!r}")

    date_part, time_part = parse_and_format_date(time_str)

    timestamp_str = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    unique_task_name = f"{script_name}_{timestamp_str}"

    final_cmd = (
        f'schtasks /create /TN "{unique_task_name}" /SC once '
        f'/ST {time_part} /SD {date_part} '
        f'/TR "{os.path.join(scripts_path, script_name + ".bat")} {arg_str}" /f'
    )
    state["schtasks_command"] = final_cmd
    print(f"[DEBUG] Final command after date fix: {final_cmd}")
    print(f"[DEBUG] windows_crontab_translator duration: {time.time() - start_time:.2f} seconds\n")
    return state

In [8]:
def run_crontab_command(state: AgentState) -> AgentState:
    print("[DEBUG] Entering run_crontab_command node...")
    start_time = time.time()
    cmd = state["schtasks_command"]
    print(f"[DEBUG] Command to be run: {cmd}")
    subprocess.run(cmd, shell=True)
    final_text = f"Scheduled with command: {cmd}"
    state["result"] = final_text
    state["messages"] = state["messages"] + [AIMessage(content=final_text)]
    print(f"[DEBUG] run_crontab_command finished. Output: {final_text}")
    print(f"[DEBUG] run_crontab_command duration: {time.time() - start_time:.2f} seconds\n")
    return state

In [9]:
workflow = StateGraph(AgentState)
workflow.add_node("time_extractor", time_extractor)
workflow.add_node("script_parser", script_parser)
workflow.add_node("windows_crontab_translator", windows_crontab_translator)
workflow.add_node("run_crontab_command", run_crontab_command)
workflow.add_edge("time_extractor", "script_parser")
workflow.add_edge("script_parser", "windows_crontab_translator")
workflow.add_edge("windows_crontab_translator", "run_crontab_command")
workflow.add_edge("run_crontab_command", END)
workflow.set_entry_point("time_extractor")
graph = workflow.compile()

In [10]:
user_input = "Hey, please run the alert in 2 minutes from now"
input_data = {"messages": [HumanMessage(content=user_input)]}
graph.invoke(input=input_data)

[DEBUG] Entering time_extractor node...
[DEBUG] time_extractor finished. LLM Output: '2025-01-18 13:52 CET'
[DEBUG] time_extractor duration: 1.02 seconds

[DEBUG] Entering script_parser node...
[DEBUG] script_parser finished. Structured LLM Output: script_name='alert_script' script_args=['--run-in', '2', 'minutes']
[DEBUG] script_parser duration: 0.80 seconds

[DEBUG] Entering windows_crontab_translator node...
[DEBUG] windows_crontab_translator finished. LLM Output: 'schtasks /create /TN "alert_script" /SC once /ST 13:52 /SD 01/18/2025 /TR "C:\\scripts\\alert_script.bat --run-in 2 minutes" /f'
[DEBUG] Final command after date fix: schtasks /create /TN "alert_script_20250118_135043" /SC once /ST 13:52 /SD 18.01.2025 /TR "c:\Users\User\Desktop\ChatGPTTask\scripts\alert_script.bat --run-in 2 minutes" /f
[DEBUG] windows_crontab_translator duration: 2.19 seconds

[DEBUG] Entering run_crontab_command node...
[DEBUG] Command to be run: schtasks /create /TN "alert_script_20250118_135043" /SC 

{'messages': [HumanMessage(content='Hey, please run the alert in 2 minutes from now', additional_kwargs={}, response_metadata={}, id='276459da-d7c5-4e55-86a0-2151f0bf3a1c'),
  AIMessage(content='Scheduled with command: schtasks /create /TN "alert_script_20250118_135043" /SC once /ST 13:52 /SD 18.01.2025 /TR "c:\\Users\\User\\Desktop\\ChatGPTTask\\scripts\\alert_script.bat --run-in 2 minutes" /f', additional_kwargs={}, response_metadata={}, id='edb341ce-5d9e-4a1b-bc7e-0a3c2801441b')],
 'extracted_time': '2025-01-18 13:52 CET',
 'script_name': 'alert_script',
 'script_args': ['--run-in', '2', 'minutes'],
 'schtasks_command': 'schtasks /create /TN "alert_script_20250118_135043" /SC once /ST 13:52 /SD 18.01.2025 /TR "c:\\Users\\User\\Desktop\\ChatGPTTask\\scripts\\alert_script.bat --run-in 2 minutes" /f',
 'result': 'Scheduled with command: schtasks /create /TN "alert_script_20250118_135043" /SC once /ST 13:52 /SD 18.01.2025 /TR "c:\\Users\\User\\Desktop\\ChatGPTTask\\scripts\\alert_script